In [1]:
import pandas as pd
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1
)

In [ ]:
# llm.invoke("What is the capital of France?")

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field

from typing import TypedDict

class AgentState(TypedDict):
    user_query: str
    explanation: str
    examples: str
    final_answer: str


# class AgentState(BaseModel):
#     user_query: str = Field(..., description="Original User Question")
#     explanation: Optional[str] = Field(None, description="Detailed Answer to the User Question")
#     examples: Optional[str] = Field(None, description="Relevant Examples Illustrating the Answer")
#     final_answer: Optional[str] = Field(None, description="Concise Final Answer for the User")

### Node 1 : Explanation

In [ ]:
@traceable(name="Explain Node")
def explain_concept(state: AgentState):
    prompt = f"""
    Explain the following topic in simple beginner-friendly terms:

    Topic: {state.user_query}
    """
    response = llm.invoke(prompt)

    return {"explanation": response.content}


### Node 2 - Example

In [ ]:
@traceable(name="Example Node")
def generate_examples(state: AgentState):
    prompt = f"""
    Based on this explanation:

    {state.explanation}

    Provide 3 real-world examples.
    """
    response = llm.invoke(prompt)

    return {"examples": response.content}


### Node 3 — Summary Step

In [ ]:
@traceable(name="Summarize Node")
def summarize_all(state: AgentState):
    prompt = f"""
    Explanation:
    {state.explanation}

    Examples:
    {state.examples}

    Create a short summary for revision.
    """
    response = llm.invoke(prompt)

    return {"final_answer": response.content}


### Agent Graph

In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("explain", explain_concept)
workflow.add_node("examples", generate_examples)
workflow.add_node("summary", summarize_all)

workflow.set_entry_point("explain")
workflow.add_edge("explain", "examples")
workflow.add_edge("examples", "summary")
workflow.add_edge("summary", END)

graph = workflow.compile()


### Run Agent

In [ ]:
result = graph.invoke(
    AgentState(user_query="What is overfitting in machine learning?")
)

In [ ]:
print(result["final_answer"])

In [ ]:
print(result["explanation"])

In [ ]:
print(result["examples"])